In [7]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
from datetime import datetime, timedelta

PROJECT_ROOT = r'D:\Supply-Chain-Efficiency-Analytics'  
DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')
DASHBOARD_DIR = os.path.join(PROJECT_ROOT, 'dashboards')

# ============================================
# 第一步：数据加载与初步清洗
# ============================================
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv'))
suppliers = pd.read_csv(os.path.join(DATA_DIR, 'suppliers.csv'))
purchase_orders = pd.read_csv(os.path.join(DATA_DIR, 'purchase_orders.csv'))
inbound_records = pd.read_csv(os.path.join(DATA_DIR, 'inbound_records.csv'))
inventory = pd.read_csv(os.path.join(DATA_DIR, 'inventory.csv'))
sales_orders = pd.read_csv(os.path.join(DATA_DIR, 'sales_orders.csv'))

# 日期转换
inventory['last_updated'] = pd.to_datetime(inventory['last_updated'])
sales_orders['sales_order_date'] = pd.to_datetime(sales_orders['sales_order_date'])
purchase_orders['purchase_order_date'] = pd.to_datetime(purchase_orders['purchase_order_date'])

# ============================================
# 第二步：全 SKU 库存天数热力图（全局扫描）
# ============================================

# 2.1 按月份聚合销售数据
sales_orders['year_month'] = sales_orders['sales_order_date'].dt.to_period('M')
monthly_sales = sales_orders.groupby(['product_id', 'year_month']).agg({
    'quantity_sold': 'sum'
}).reset_index()
monthly_sales['year_month'] = monthly_sales['year_month'].dt.to_timestamp()

# 2.2 按月份聚合库存数据
inventory['year_month'] = inventory['last_updated'].dt.to_period('M')
monthly_inventory = inventory.groupby(['product_id', 'year_month']).agg({
    'quantity': 'mean',  # 月均库存
    'warehouse_location': lambda x: ', '.join(x.unique())  # 仓库位置
}).reset_index()
monthly_inventory['year_month'] = monthly_inventory['year_month'].dt.to_timestamp()

# 2.3 合并库存与销售数据，计算库存天数
inventory_sales = monthly_inventory.merge(
    monthly_sales,
    on=['product_id', 'year_month'],
    how='left'
)

# 计算日均销量与库存天数
inventory_sales['daily_sales'] = inventory_sales['quantity_sold'].fillna(0) / 30
inventory_sales['inventory_days'] = inventory_sales['quantity'] / (inventory_sales['daily_sales'] + 0.01)  # 避免除 0
inventory_sales.loc[inventory_sales['daily_sales'] == 0, 'inventory_days'] = 999  # 无销量标记为 999+

# 2.4 关联产品信息
inventory_sales = inventory_sales.merge(
    products[['product_id', 'sku', 'product_name', 'category', 'safety_stock']],
    on='product_id'
)

# 2.5 生成热力图数据（宽表格式）
heatmap_pivot = inventory_sales.pivot_table(
    index='sku',
    columns='year_month',
    values='inventory_days',
    aggfunc='mean'
)

# 2.6 可视化热力图（全 SKU）
fig_heatmap_full = px.imshow(
    heatmap_pivot,
    labels=dict(x="时间", y="SKU", color="库存天数"),
    aspect="auto",
    title="全 SKU 月度库存天数热力图（0-100=正常，100-500=积压，500+=严重）",
    color_continuous_scale=['lightblue', 'orange', 'red'],
    color_continuous_midpoint=250,
    height=600,
    width=1000
)
fig_heatmap_full.update_layout(
    xaxis_title="时间",
    yaxis_title="SKU",
    font=dict(size=9),
    margin=dict(l=80, r=50, t=80, b=80),
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickfont=dict(size=8))
)
fig_heatmap_full.update_xaxes(showgrid=False)
fig_heatmap_full.update_yaxes(showgrid=False)

# ============================================
# 第三步：异常 SKU 筛选
# ============================================

# 3.1 规则 1：连续 3 个月库存天数 > 500
sku_continuous_high = []
for sku in heatmap_pivot.index:
    row = heatmap_pivot.loc[sku]
    # 找连续 3 个月 > 500 的情况
    high_months = (row > 500).astype(int)
    consecutive = 0
    max_consecutive = 0
    for val in high_months:
        if val == 1:
            consecutive += 1
            max_consecutive = max(max_consecutive, consecutive)
        else:
            consecutive = 0
    if max_consecutive >= 3:
        sku_continuous_high.append(sku)

# 3.2 规则 2：单月环比增长 > 200%
sku_rapid_growth = []
for sku in heatmap_pivot.index:
    row = heatmap_pivot.loc[sku].dropna()
    if len(row) > 1:
        pct_change = row.pct_change()
        if (pct_change > 2).any():  # 增长超过 200%
            sku_rapid_growth.append(sku)

# 3.3 规则 3：库存天数超出同 category 均值 3 倍
sku_category_outlier = []
category_mean = inventory_sales.groupby('category')['inventory_days'].mean()
for _, row in inventory_sales.iterrows():
    if row['inventory_days'] > category_mean[row['category']] * 3:
        if row['sku'] not in sku_category_outlier:
            sku_category_outlier.append(row['sku'])

# 合并异常 SKU 清单（去重）
abnormal_skus = list(set(sku_continuous_high + sku_rapid_growth + sku_category_outlier))
abnormal_df = inventory_sales[inventory_sales['sku'].isin(abnormal_skus)][
    ['sku', 'product_name', 'category', 'safety_stock', 'quantity', 'inventory_days', 'warehouse_location']
].drop_duplicates()

print(f"[异常 SKU 数量] 总计：{len(abnormal_skus)}")
print(f"  - 规则 1（连续高位）：{len(sku_continuous_high)}")
print(f"  - 规则 2（快速增长）：{len(sku_rapid_growth)}")
print(f"  - 规则 3（同 category 离群）：{len(sku_category_outlier)}")


# ============================================
# 第四步：异常 SKU 深度分析（折线图）
# ============================================

# 4.1 日粒度库存数据整合
inventory_daily = inventory.merge(
    products[['product_id', 'sku', 'product_name', 'category']],
    on='product_id'
).copy()

print(f"[库存日度数据] 共 {len(inventory_daily)} 条记录，涉及 {inventory_daily['product_id'].nunique()} 个产品")

# 4.2 日粒度销售数据
sales_daily = sales_orders.groupby(['product_id', 'sales_order_date']).agg({
    'quantity_sold': 'sum'
}).reset_index()
sales_daily.rename(columns={'sales_order_date': 'last_updated'}, inplace=True)

print(f"[销售日度数据] 共 {len(sales_daily)} 条记录")

# 4.3 滚动计算 7 日平均销量（平滑波动）
sales_daily['daily_sales_7d'] = sales_daily.groupby('product_id')['quantity_sold'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

# 4.4 合并库存与销售
inventory_daily_merged = inventory_daily.merge(
    sales_daily[['product_id', 'last_updated', 'daily_sales_7d', 'quantity_sold']],
    on=['product_id', 'last_updated'],
    how='left'
).fillna(0)

# 计算日度库存天数
inventory_daily_merged['inventory_days_daily'] = inventory_daily_merged['quantity'] / (
    inventory_daily_merged['daily_sales_7d'] + 0.01
)

print(f"[合并后数据] 共 {len(inventory_daily_merged)} 条记录")

# 4.5 对于异常 SKU，生成单 SKU 分析图
def create_single_sku_chart(sku, days_lookback=None):
    """单 SKU 库存天数 + 库存数量双轴图"""
    # 获取 product_id
    sku_all_data = inventory_daily_merged[inventory_daily_merged['sku'] == sku]
    
    if sku_all_data.empty:
        print(f"  ⚠ SKU {sku} 在库存数据中找不到")
        return None
    
    product_id = sku_all_data['product_id'].iloc[0]
    
    # 如果指定了 days_lookback，则筛选；否则使用所有数据
    if days_lookback is not None:
        sku_data = sku_all_data[
            sku_all_data['last_updated'] >= pd.Timestamp.now() - timedelta(days=days_lookback)
        ].sort_values('last_updated').copy()
    else:
        # 使用所有可用数据
        sku_data = sku_all_data.sort_values('last_updated').copy()
    
    if sku_data.empty:
        print(f"  ⚠ SKU {sku} 无可用数据")
        return None
    
    print(f"  ✓ SKU {sku}: 获取 {len(sku_data)} 条数据（{sku_data['last_updated'].min().date()} 至 {sku_data['last_updated'].max().date()}）")
    
    product_name = sku_data['product_name'].iloc[0]
    date_range = (sku_data['last_updated'].max() - sku_data['last_updated'].min()).days
    
    # 创建双轴图
    fig = make_subplots(
        specs=[[{"secondary_y": True}]],
        subplot_titles=(f"SKU: {sku} - {product_name}",)
    )
    
    # 左轴：库存天数
    fig.add_trace(
        go.Scatter(
            x=sku_data['last_updated'],
            y=sku_data['inventory_days_daily'].fillna(0),
            name='库存天数',
            line=dict(color='#1f77b4', width=2),
            mode='lines+markers',
            hovertemplate='<b>日期</b>: %{x|%Y-%m-%d}<br><b>库存天数</b>: %{y:.1f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # 右轴：库存数量
    fig.add_trace(
        go.Scatter(
            x=sku_data['last_updated'],
            y=sku_data['quantity'].fillna(0),
            name='库存数量',
            line=dict(color='#2ca02c', width=2, dash='dash'),
            mode='lines+markers',
            hovertemplate='<b>日期</b>: %{x|%Y-%m-%d}<br><b>库存数量</b>: %{y:.0f}<extra></extra>'
        ),
        secondary_y=True
    )
    
    # 标注采购订单
    try:
        po_sku = purchase_orders[purchase_orders['product_id'] == product_id]
        po_sku_recent = po_sku[po_sku['purchase_order_date'] >= sku_data['last_updated'].min()]
        
        if not po_sku_recent.empty:
            for _, po in po_sku_recent.iterrows():
                fig.add_vline(
                    x=po['purchase_order_date'],
                    line_dash="dot",
                    line_color="red",
                    annotation_text=f"采购: {po['quantity']}",
                    annotation_position="top left"
                )
    except Exception as e:
        print(f"    (标注采购订单失败: {str(e)})")
    
    fig.update_layout(
        title=f"SKU: {sku} - {product_name} 库存趋势分析（共 {date_range} 天数据）",
        xaxis_title="时间",
        height=500,
        width=1000,
        hovermode='x unified',
        margin=dict(l=80, r=80, t=100, b=80),
        showlegend=True
    )
    
    fig.update_xaxes(title_text="时间", tickangle=-45)
    fig.update_yaxes(title_text="库存天数 (天)", secondary_y=False)
    fig.update_yaxes(title_text="库存数量 (件)", secondary_y=True)
    
    return fig


# 为异常 SKU 生成图表 - 改为使用所有可用数据
print("\n[生成异常 SKU 分析图表]")
abnormal_sku_charts = {}
for idx, sku in enumerate(abnormal_skus[:5], 1):
    print(f"处理异常 SKU {idx}/5: {sku}")
    fig = create_single_sku_chart(sku, days_lookback=None)  # 使用 None 表示所有数据
    if fig:
        abnormal_sku_charts[sku] = fig
        print(f"    ✅ 图表生成成功")
    else:
        print(f"  ✗ SKU {sku} 图表生成失败")

print(f"\n[完成] 共生成 {len(abnormal_sku_charts)} 个异常 SKU 图表")

# 4.6 同 category 异常 SKU 对比图 - 使用所有可用数据
def create_category_comparison_chart(category, abnormal_skus_in_category):
    """同 category 异常 SKU 周度库存天数对比"""
    cat_data = inventory_daily_merged[
        (inventory_daily_merged['category'] == category) &
        (inventory_daily_merged['sku'].isin(abnormal_skus_in_category))
    ].copy()
    
    if cat_data.empty:
        print(f"  ⚠ 品类 {category} 无数据")
        return None
    
    # 按周聚合
    cat_data['year_week'] = cat_data['last_updated'].dt.to_period('W')
    cat_weekly = cat_data.groupby(['sku', 'year_week']).agg({
        'inventory_days_daily': 'mean'
    }).reset_index()
    cat_weekly['year_week'] = cat_weekly['year_week'].dt.to_timestamp()
    
    if cat_weekly.empty:
        print(f"  ⚠ 品类 {category} 周度聚合后无数据")
        return None
    
    print(f"  ✓ 品类 {category}: 获取 {len(cat_weekly)} 条周度数据，涉及 {cat_weekly['sku'].nunique()} 个 SKU")
    
    fig = px.line(
        cat_weekly,
        x='year_week',
        y='inventory_days_daily',
        color='sku',
        title=f"品类：{category} - 异常 SKU 周度库存天数对比",
        labels={'year_week': '周份', 'inventory_days_daily': '库存天数 (天)', 'sku': 'SKU'},
        height=500,
        width=1000,
        markers=True
    )
    
    fig.update_layout(
        margin=dict(l=80, r=80, t=100, b=80),
        hovermode='x unified',
        showlegend=True,
        xaxis_tickangle=-45
    )
    
    return fig

# 生成品类对比图表
print("\n[生成品类对比分析图表]")
category_comparison_charts = {}
for category in abnormal_df['category'].unique():
    cat_abnormal_skus = abnormal_df[abnormal_df['category'] == category]['sku'].tolist()
    if len(cat_abnormal_skus) > 0:  # 改为 > 0，允许单个 SKU 的品类也生成
        print(f"处理品类：{category}（{len(cat_abnormal_skus)} 个异常 SKU）")
        fig = create_category_comparison_chart(category, cat_abnormal_skus)
        if fig:
            category_comparison_charts[category] = fig
            print(f"    ✅ 图表生成成功")
        else:
            print(f"  ✗ 品类 {category} 图表生成失败")
    else:
        print(f"跳过品类：{category}（无异常 SKU）")

print(f"\n[完成] 共生成 {len(category_comparison_charts)} 个品类对比图表")

# ============================================
# 第五步：积压原因分析
# ============================================

def analyze_abnormal_sku(sku):
    """分析单个异常 SKU 的积压原因"""
    sku_info = abnormal_df[abnormal_df['sku'] == sku].iloc[0] if sku in abnormal_df['sku'].values else None
    if sku_info is None:
        return {}
    
    product_id = inventory_sales[inventory_sales['sku'] == sku]['product_id'].iloc[0]
    
    # 采购量 vs 销售量
    po_total = purchase_orders[purchase_orders['product_id'] == product_id]['quantity'].sum()
    sales_total = sales_orders[sales_orders['product_id'] == product_id]['quantity_sold'].sum()
    
    # 最近 30 天分析
    last_30_days = pd.Timestamp.now() - timedelta(days=30)
    po_recent = purchase_orders[
        (purchase_orders['product_id'] == product_id) &
        (purchase_orders['purchase_order_date'] >= last_30_days)
    ]['quantity'].sum()
    sales_recent = sales_orders[
        (sales_orders['product_id'] == product_id) &
        (sales_orders['sales_order_date'] >= last_30_days)
    ]['quantity_sold'].sum()
    
    # 供应商评级
    supplier_ids = purchase_orders[purchase_orders['product_id'] == product_id]['supplier_id'].unique()
    supplier_ratings = suppliers[suppliers['supplier_id'].isin(supplier_ids)]['rating'].mean()
    
    analysis = {
        'sku': sku,
        'product_name': sku_info['product_name'],
        'category': sku_info['category'],
        'safety_stock': sku_info['safety_stock'],
        'current_inventory': sku_info['quantity'],
        'current_inventory_days': sku_info['inventory_days'],
        'warehouse_location': sku_info['warehouse_location'],
        'total_po_qty': po_total,
        'total_sales_qty': sales_total,
        'po_to_sales_ratio': po_total / (sales_total + 0.01),
        'recent_30d_po': po_recent,
        'recent_30d_sales': sales_recent,
        'avg_supplier_rating': supplier_ratings,
        'reason': []
    }
    
    # 原因判断
    if analysis['po_to_sales_ratio'] > 1.5:
        analysis['reason'].append('采购过量（采购量 / 销售量 > 1.5）')
    
    if sales_recent < sku_info['safety_stock'] * 0.3:
        analysis['reason'].append('销量低迷（最近 30 天销售量 < 安全库存的 30%）')
    
    if sku_info['safety_stock'] > sales_total / 12:
        analysis['reason'].append('安全库存设置过高')
    
    if analysis['avg_supplier_rating'] < 3.5:
        analysis['reason'].append('供应商评级低（可能导致交货延迟）')
    
    if analysis['recent_30d_po'] > analysis['recent_30d_sales'] * 2:
        analysis['reason'].append('最近采购突增，销售未跟上')
    
    if not analysis['reason']:
        analysis['reason'] = ['其他原因']
    
    return analysis

abnormal_analysis = [analyze_abnormal_sku(sku) for sku in abnormal_skus]

# ============================================
# 第六步：健康度统计与优化建议
# ============================================

# 库存健康度分类
inventory_health = inventory_sales.copy()
inventory_health['health_status'] = inventory_health['inventory_days'].apply(
    lambda x: '严重积压（500+天）' if x >= 500 else ('积压（100-500天）' if x >= 100 else '正常（<100天）')
)

health_summary = inventory_health.groupby('category').agg({
    'sku': 'nunique'
}).reset_index()
health_summary.columns = ['category', 'total_sku']

for status in ['正常（<100天）', '积压（100-500天）', '严重积压（500+天）']:
    health_summary[status] = health_summary['category'].apply(
        lambda cat: len(
            inventory_health[
                (inventory_health['category'] == cat) &
                (inventory_health['health_status'] == status)
            ]['sku'].unique()
        )
    )

# 优化建议生成
optimization_suggestions = []
for analysis in abnormal_analysis:
    sku = analysis['sku']
    reason_str = '；'.join(analysis['reason'])
    
    suggestions = []
    if '采购过量' in reason_str:
        suggestions.append(f"暂停或减少对此 SKU 的采购，建议周期性评估最小采购量")
    
    if '销量低迷' in reason_str:
        suggestions.append(f"启动促销活动或清仓促销，加快销售速度")
    
    if '安全库存过高' in reason_str:
        suggestions.append(f"将安全库存从 {analysis['safety_stock']} 降低至 {int(analysis['safety_stock'] * 0.6)}")
    
    if '供应商评级低' in reason_str:
        supplier_ids = purchase_orders[purchase_orders['product_id'] == inventory_sales[inventory_sales['sku'] == sku]['product_id'].iloc[0]]['supplier_id'].unique()
        low_rating_suppliers = suppliers[
            (suppliers['supplier_id'].isin(supplier_ids)) &
            (suppliers['rating'] < 3.5)
        ]['supplier_name'].tolist()
        if low_rating_suppliers:
            suggestions.append(f"更换供应商，考虑替代供应商以缩短交货周期（当前低评级供应商：{', '.join(low_rating_suppliers)}）")
    
    if '最近采购突增' in reason_str:
        suggestions.append(f"已有大量在途库存，建议暂停新采购，等待销售消化")
    
    optimization_suggestions.append({
        'sku': sku,
        'product_name': analysis['product_name'],
        'category': analysis['category'],
        'primary_reason': analysis['reason'][0] if analysis['reason'] else '未知',
        'suggestions': suggestions if suggestions else ['继续监控库存变动']
    })

# ============================================
# 第七步：保存结果与可视化
# ============================================

# 保存所有结果到 Pickle
eda_inventory_analysis = {
    'fig_heatmap_full': fig_heatmap_full,
    'abnormal_sku_charts': abnormal_sku_charts,
    'category_comparison_charts': category_comparison_charts,
    'abnormal_sku_list': abnormal_df,
    'health_summary': health_summary,
    'abnormal_analysis': abnormal_analysis,
    'optimization_suggestions': optimization_suggestions
}

pickle_path = os.path.join(DASHBOARD_DIR, 'inventory_analysis_results.pkl')
os.makedirs(DASHBOARD_DIR, exist_ok=True)
with open(pickle_path, 'wb') as f:
    pickle.dump(eda_inventory_analysis, f)

print(f"\n[保存完成] 分析结果已保存到：{pickle_path}")

# 输出健康度摘要
print("\n" + "="*60)
print("库存健康度摘要")
print("="*60)
print(health_summary.to_string(index=False))

# 输出异常 SKU 摘要
print("\n" + "="*60)
print(f"异常 SKU 清单（共 {len(abnormal_skus)} 个）")
print("="*60)
for idx, item in enumerate(optimization_suggestions[:10], 1):
    print(f"\n{idx}. SKU: {item['sku']} - {item['product_name']}")
    print(f"   品类：{item['category']}")
    print(f"   主要原因：{item['primary_reason']}")
    print(f"   建议：")
    for sug in item['suggestions']:
        print(f"     • {sug}")


[异常 SKU 数量] 总计：18
  - 规则 1（连续高位）：0
  - 规则 2（快速增长）：13
  - 规则 3（同 category 离群）：5
[库存日度数据] 共 1000 条记录，涉及 200 个产品
[销售日度数据] 共 2943 条记录
[合并后数据] 共 1000 条记录

[生成异常 SKU 分析图表]
处理异常 SKU 1/5: SKU1035
  ✓ SKU SKU1035: 获取 5 条数据（2023-11-08 至 2023-12-01）
    ✅ 图表生成成功
处理异常 SKU 2/5: SKU1018
  ✓ SKU SKU1018: 获取 5 条数据（2023-11-11 至 2023-12-01）
    ✅ 图表生成成功
处理异常 SKU 3/5: SKU1188
  ✓ SKU SKU1188: 获取 5 条数据（2023-11-04 至 2023-12-01）
    ✅ 图表生成成功
处理异常 SKU 4/5: SKU1114
  ✓ SKU SKU1114: 获取 5 条数据（2023-11-12 至 2023-12-01）
    ✅ 图表生成成功
处理异常 SKU 5/5: SKU1117
  ✓ SKU SKU1117: 获取 5 条数据（2023-11-08 至 2023-12-01）
    ✅ 图表生成成功

[完成] 共生成 5 个异常 SKU 图表

[生成品类对比分析图表]
处理品类：C（8 个异常 SKU）
  ✓ 品类 C: 获取 19 条周度数据，涉及 5 个 SKU
    ✅ 图表生成成功
处理品类：A（13 个异常 SKU）
  ✓ 品类 A: 获取 23 条周度数据，涉及 7 个 SKU
    ✅ 图表生成成功
处理品类：B（10 个异常 SKU）
  ✓ 品类 B: 获取 21 条周度数据，涉及 6 个 SKU
    ✅ 图表生成成功

[完成] 共生成 3 个品类对比图表

[保存完成] 分析结果已保存到：D:\Supply-Chain-Efficiency-Analytics\dashboards\inventory_analysis_results.pkl

库存健康度摘要
category  total_sku  正常（<100天）  积压（100-500天）  严重